In [3]:
import pandas as pd
import numpy as np
import os
import cv2

In [7]:
data = pd.read_hdf(r"goodcenterline.h5")

In [8]:
data = data.reset_index(drop = True)
data['loc_x'] = pd.cut(data['lat'], 1024, labels=False) #1024->8192 lat
data['loc_y'] = pd.cut(data['lon'], 1024, labels=False) #1024->8192 lon
print(data)

             lon        lat  count  loc_x  loc_y
0     121.658977  38.963762    0.0    603    708
1     121.660975  38.956770    0.0    601    709
2     121.666093  38.961140    0.0    602    710
3     121.672835  38.964136    0.0    603    711
4     121.680825  38.966009    0.0    603    712
...          ...        ...    ...    ...    ...
4685  121.106755  37.960340  119.0    356    607
4686  121.099957  37.961991  119.0    356    606
4687  121.095463  37.967734  119.0    357    605
4688  121.088466  37.969422  119.0    358    604
4689  121.073932  37.996414  119.0    365    601

[4690 rows x 5 columns]


lon;lat

In [ ]:
# Filter data within the error range.
def filter_loc(series):
    x = series['lon']
    y = series['lat']
    if 117.881<=x<=118.197 and 38.337<=y<=38.457:
        return 1
    elif 118.266<=x<=119.163 and 38.369<=y<=38.891:
        return 1
    elif 121.749<=x<=122.143 and 38.581<=y<=38.806:
        return 1
    else:
        return 0
data['filter']= data.apply(filter_loc,axis =1)
data

,lon,lat,count,loc_x,loc_y,filter
0,119.037975,38.428636,0.0,1880,923,1
1,119.097831,38.505603,1.0,1956,965,1
2,123.238012,37.556155,2.0,1022,3868,0
3,123.238512,37.552035,2.0,1018,3869,0
4,123.242507,37.552170,2.0,1018,3872,0
...,...,...,...,...,...,...
9370,118.119607,38.379060,1366.0,1831,279,1
9371,118.060826,38.370739,1367.0,1823,238,1
9372,120.251770,39.173813,1368.0,2613,1774,0
9373,119.102977,38.463482,1369.0,1914,969,1


In [ ]:
result = data[data['filter'] ==0]
result = result.drop(['filter'],axis = 1)

In [ ]:
result = result.reset_index(drop=True)
result

,lon,lat,count,loc_x,loc_y
0,123.238012,37.556155,2.0,1022,3868
1,123.238512,37.552035,2.0,1018,3869
2,123.242507,37.552170,2.0,1018,3872
3,123.243006,37.548050,2.0,1014,3872
4,123.243505,37.543930,2.0,1010,3872
...,...,...,...,...,...
7351,120.273649,39.210205,1361.0,2649,1790
7352,120.270798,39.214949,1361.0,2653,1788
7353,120.268770,39.220380,1361.0,2659,1786
7354,120.251770,39.173813,1368.0,2613,1774


In [ ]:
def flip180(arr):
    new_arr = arr.reshape(arr.size)
    new_arr = new_arr[::-1]
    new_arr = new_arr.reshape(arr.shape)
    return new_arr

def flip90_left(arr):
    new_arr = np.transpose(arr)
    new_arr = new_arr[::-1]
    return new_arr

def flip90_right(arr):
    new_arr = arr.reshape(arr.size)
    new_arr = new_arr[::-1]
    new_arr = new_arr.reshape(arr.shape)
    new_arr = np.transpose(new_arr)[::-1]
    return new_arr


In [9]:
# centerline.png
import cv2
ship_group = data.groupby('count') #result
center_line = np.zeros((1024,1024), dtype=np.uint8) #1024->8192
for count,group in ship_group:
    group = group.reset_index(drop = True)
    temp = np.zeros((1024,1024), dtype=np.uint8) #1024->8192
    for i in range(len(group)):
        if i != len(group)-1:
            now_x = group.loc[i,'loc_x']
            now_y = group.loc[i,'loc_y']
            next_x = group.loc[i+1,'loc_x']
            next_y = group.loc[i+1,'loc_y']
            if now_x == next_x and now_y == next_y:
                continue
            else:
                cv2.line(temp, (now_x, now_y), (next_x, next_y), 255, 1, lineType=cv2.LINE_8)
    center_line = cv2.add(center_line,temp)
#center_line = flip180(center_line)
cv2.imwrite(r'goodcenterline.png', center_line)

True

In [ ]:
# Draw region.png
centerline_img = cv2.imread(r'cycleGAN_input/goodcenterline256.png', cv2.IMREAD_GRAYSCALE)
centerline_pixels = np.where(centerline_img == 255)
H, W = centerline_img.shape
radius = 2
region_img = np.zeros(centerline_img.shape, dtype=np.uint8)
for i, j in list(zip(centerline_pixels[0], centerline_pixels[1])):
    for y in range(max(i-radius, 0), min(i+radius+1, H)):
        for x in range(max(j-radius, 0), min(j+radius+1, W)):
            region_img[y, x] = 255
cv2.imwrite(os.path.join(r'cycleGAN_input/goodregion256.png'), region_img)

True